In [2]:
import pandas as pd

data = pd.read_csv('news_sample.csv', header = 0)

In [3]:
from cleantext import clean

def clean_text(s):
    return clean(s,lower=True,                     # lowercase text
        no_urls=True,                  # replace all URLs with a special token
        no_emails=True,                # replace all email addresses with a special token
        no_numbers=True,               # replace all numbers with a special token
        replace_with_url="<URL>",
        replace_with_email="<EMAIL>",
        replace_with_number="<NUM>",
        lang="en"                   
    )

data['cleaned_content'] = data['content'].apply(clean_text)

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [9]:
import nltk
#nltk.download('punkt')
from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()
data['tokenized_content'] = data['cleaned_content'].apply(tokenizer.tokenize)

In [12]:
from nltk.corpus import stopwords
#nltk.download('stopwords')

def remove_stopwords(s):
    return [w for w in s if w not in stopwords.words('english')]
data['tokenized_content'] = data['tokenized_content'].apply(remove_stopwords)

In [13]:
data['tokenized_content']

0      [sometimes, power, christmas, make, wild, wond...
1      [awakening, <, num, >, strands, dna, ``, recon...
2      [never, hike, alone, :, friday, 13th, fan, fil...
3      [rare, shark, caught, ,, scientists, left, blu...
4      [donald, trump, unnerving, ability, ability, c...
                             ...                        
245    [prison, rahm, ,, god, 's, work, many, others,...
246    [<, num, >, useful, items, tiny, home, headlin...
247    [former, cia, director, michael, hayden, said,...
248    [antonio, sabato, jr., says, hollywood, 's, li...
249    [former, u.s., president, bill, clinton, monda...
Name: tokenized_content, Length: 250, dtype: object